# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents_NEW.txt"
sentences = MySentences(infile)



### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

815775

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

taxes_communale


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

2

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

TypeError: 'MySentences' object is not subscriptable

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

TypeError: 'MySentences' object is not subscriptable

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], ['bruxelles'], ['bulletin', 'ires', '8eanas'], ['dl', '!'], ['conseil'], ['communal'], ['annee'], ['1847', '.'], ['au'], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin'], ['conseil'], ['aes'], ['seances'], ['communal', '.'], ['annee'], ['1847', '.'], ['bruxelles', ','], ['imprimerie'], ['d', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], ['1', '84', '8'], ['de', '!'], ['du', 'consei'], ['dibi'], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel_hospice', 'pour', 'les', 'av'], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des', 'taxes', 'communale', "'"], ['bieniaance', 'eldeseianv'], ['il', 'est', 'donne', 'communie', ';'], ['mandant', 'le', 'o', 'p', 'fa'], ['gnant

## Entrainement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 55.3 s, sys: 1.38 s, total: 56.7 s
Wall time: 26.4 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [21]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [22]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [23]:
model.wv["bruxelles"]

array([-0.13642886, -1.4456534 , -0.97562164, -0.05776419,  1.0988746 ,
       -0.71282053,  1.7406076 , -0.1941411 , -0.9376089 , -0.97631806,
       -1.5891856 , -1.3989004 ,  1.3638022 , -1.4436193 ,  0.9063768 ,
        0.628425  , -1.0356606 , -1.3662843 , -2.5821197 , -2.5133114 ,
        0.60106856, -1.2254366 ,  2.321621  , -2.823328  , -2.8058925 ,
       -1.5413907 , -2.3633897 , -3.2034652 , -2.1907747 ,  1.7652597 ,
       -4.0205355 ,  0.0658287 ], dtype=float32)

### Calculer la similarité entre deux termes

In [27]:
model.wv.similarity("ecole", "commune")

0.40205777

### Chercher les mots les plus proches d'un terme donné

In [26]:
model.wv.most_similar("eglise", topn=20)

[('impasse', 0.7711295485496521),
 ('etang', 0.7648087739944458),
 ('agrandissement', 0.7499045729637146),
 ('avenue', 0.7415223121643066),
 ('escalier', 0.7270488142967224),
 ('eglise_sainte', 0.7217838764190674),
 ('erection', 0.7184381484985352),
 ('achevement', 0.7167012691497803),
 ('incorporation', 0.7155272364616394),
 ('ecole', 0.6996915936470032),
 ('entree', 0.6947248578071594),
 ('ancien_bassin', 0.6914601922035217),
 ('etablissement', 0.6906641721725464),
 ('hopital', 0.6904030442237854),
 ('ancienne_caserne', 0.685377299785614),
 ('esplanade', 0.6849141716957092),
 ('paroisse', 0.6821412444114685),
 ('entrepot', 0.6789653897285461),
 ('armee', 0.676539957523346),
 ('hopital_st_.-_jean', 0.6757571697235107)]

### Faire des recherches complexes à travers l'espace vectoriel

In [53]:
print(model.wv.most_similar(positive=['taxe', 'alimentation', 'nourriture'], negative=['agriculture', 'belgique']))

[('valeur', 0.6542356014251709), ('deces_presente', 0.6290267109870911), ('reduction', 0.61492919921875), ('difference_entre', 0.6094015836715698), ('base', 0.6020293235778809), ('consommation', 0.5885253548622131), ('houille', 0.5872054696083069), ('diminution', 0.5847838521003723), ('difference', 0.5839007496833801), ('quantite', 0.5825128555297852)]
